In [14]:
# -*- coding: utf-8 -*-
import numpy as np
import pandas as pd
import datetime
import sys
import re
import gc
import glob

import os
HOME = os.path.expanduser('~')
sys.path.append(f"{HOME}/kaggle/data_analysis/library/")
import utils
from utils import logger_func, get_categorical_features, get_numeric_features, pararell_process
logger = logger_func()
pd.set_option('max_columns', 200)
pd.set_option('max_rows', 200)

#========================================================================
# Global Variable
#========================================================================

2018-11-10 18:53:43,603 utils 353 [INFO]    [logger_func] start 


In [15]:
train = utils.read_df_pkl(path='../input/train*.p')
test = utils.read_df_pkl(path='../input/test*.p')

100%|██████████| 3/3 [00:00<00:00, 96.36it/s]


In [20]:
from keras import preprocessing
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

max_len = 100
training_samples = 200
validation_samples = 10000
max_words = 10000

texts = train['question_text'].values[:training_samples]

# 単語IDとしてもつのは最大10,000語まで
tokenizer = Tokenizer(num_words=max_words)
# 読み込んだテキストを単語に分割し、IDを登録する
tokenizer.fit_on_texts(texts)
# テキストを単語に分割し、IDに置き換える
sequences = tokenizer.texts_to_sequences(texts)
sequences

[[11, 47, 274, 275, 139, 39, 276, 22, 3, 277, 4, 1, 278],
 [9, 15, 18, 33, 279, 280, 11, 52, 15, 281, 57, 5, 99, 7, 53, 282],
 [16, 21, 140, 141, 142, 21, 140, 141, 143, 283],
 [11, 47, 284, 285, 286, 100, 1, 287, 288],
 [13, 8, 289, 290, 291, 292, 5, 3, 293, 294, 40, 144, 295, 1, 296],
 [6, 297, 298, 299, 300, 301, 28, 302, 14, 303],
 [16,
  21,
  145,
  146,
  304,
  305,
  306,
  54,
  307,
  73,
  21,
  53,
  9,
  1,
  147,
  14,
  308,
  309],
 [6, 19, 310, 26, 8, 311, 28, 312, 17, 313, 314, 148, 12, 315],
 [6, 24, 58, 3, 74, 22, 149, 316, 7, 26, 34, 11, 6, 20, 317, 101, 149, 318],
 [6,
  19,
  144,
  59,
  28,
  18,
  15,
  75,
  102,
  4,
  150,
  319,
  320,
  15,
  321,
  322,
  5,
  1,
  57,
  15,
  323,
  324,
  325,
  326,
  39,
  327,
  151,
  34,
  15,
  29,
  5,
  18,
  152,
  153,
  41,
  76,
  7,
  328,
  329,
  42,
  330,
  11,
  47,
  154,
  48],
 [2, 13, 15, 155, 43, 331],
 [11, 156, 1, 332, 333, 334],
 [2, 6, 1, 335, 336, 337, 338, 157, 14, 339, 77, 340],
 [13,
  4

In [22]:
word_index = tokenizer.word_index
word_index

{'the': 1,
 'what': 2,
 'a': 3,
 'in': 4,
 'to': 5,
 'is': 6,
 'and': 7,
 'i': 8,
 'do': 9,
 'of': 10,
 'how': 11,
 'are': 12,
 'can': 13,
 'for': 14,
 'you': 15,
 'why': 16,
 'my': 17,
 'have': 18,
 'it': 19,
 'that': 20,
 'does': 21,
 'as': 22,
 'be': 23,
 'there': 24,
 'should': 25,
 'if': 26,
 'will': 27,
 'or': 28,
 'get': 29,
 'has': 30,
 'which': 31,
 'they': 32,
 'an': 33,
 'so': 34,
 'with': 35,
 'from': 36,
 'on': 37,
 'best': 38,
 'their': 39,
 'by': 40,
 'your': 41,
 'at': 42,
 'about': 43,
 'we': 44,
 'know': 45,
 'who': 46,
 'did': 47,
 'change': 48,
 'life': 49,
 'one': 50,
 'like': 51,
 'would': 52,
 'not': 53,
 'when': 54,
 'all': 55,
 'some': 56,
 'people': 57,
 'such': 58,
 'me': 59,
 'many': 60,
 'make': 61,
 'them': 62,
 'doing': 63,
 'any': 64,
 'between': 65,
 'someone': 66,
 'her': 67,
 'into': 68,
 'he': 69,
 'most': 70,
 'his': 71,
 'work': 72,
 'but': 73,
 'thing': 74,
 'ever': 75,
 'way': 76,
 'being': 77,
 'home': 78,
 'no': 79,
 'find': 80,
 'good': 81,
 '

In [38]:
# テキスト毎に単語IDへ変換された単語リストを読み込み、単語カラムを合わせつつ次元を揃える。1テキストの単語数の上限を指定した場合は、その上限分の次元になる。
data = pad_sequences(sequences, maxlen=max_len)
labels = np.asarray(train['target'])[:training_samples]
print(data.shape)
print(labels.shape)

(200, 100)
(200,)


In [44]:
a = np.arange(10)
b = np.arange(10, 20, 1)
np.stack([a, b])

array([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14, 15, 16, 17, 18, 19]])

In [45]:
from tqdm import tqdm
EMBEDDING_FILE = "../model/glove.840B.300d/glove.840B.300d.txt"
## ここでの*arrはarrayをtubleで囲う役割になっている。listを解除する役割じゃなかったのか？謎
def get_coefs(word, *arr): return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.split(" ")) for o in tqdm(open(EMBEDDING_FILE)) )

all_embs = np.stack(embeddings_index.values())
emb_mean, emb_std = all_embs.mean(), all_embs.std()
embed_size = all_embs.shape[1]

word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
## どんな引数になってるんだ？
embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector



0it [00:00, ?it/s]

1860it [00:00, 18596.34it/s]

3727it [00:00, 18616.51it/s]

5595it [00:00, 18634.72it/s]

7461it [00:00, 18640.04it/s]

9387it [00:00, 18821.13it/s]

11251it [00:00, 18765.97it/s]

13122it [00:00, 18746.59it/s]

14992it [00:00, 18730.37it/s]

16859it [00:00, 18710.05it/s]

18734it [00:01, 18719.72it/s]

20606it [00:01, 18716.93it/s]

22472it [00:01, 18698.44it/s]

24339it [00:01, 18688.34it/s]

26214it [00:01, 18704.53it/s]

28083it [00:01, 18699.87it/s]

29959it [00:01, 18715.98it/s]

31833it [00:01, 18722.22it/s]

33704it [00:01, 18718.34it/s]

35577it [00:01, 18719.90it/s]

37447it [00:02, 18694.86it/s]

39320it [00:02, 18703.23it/s]

41191it [00:02, 18705.19it/s]

43062it [00:02, 18705.62it/s]

44933it [00:02, 18694.48it/s]

46810it [00:02, 18715.06it/s]

48689it [00:02, 18736.84it/s]

50569it [00:02, 18754.29it/s]

52445it [00:02, 18753.05it/s]

54322it [00:02, 18755.12it/s]

56200it [00:03, 18760.66it/s]

58080it [00:03, 18772.40it/s]

59958it [00:03, 18744.

404347it [00:21, 18535.49it/s]

406201it [00:21, 18533.51it/s]

408056it [00:21, 18537.28it/s]

409910it [00:22, 18532.63it/s]

411764it [00:22, 18523.33it/s]

413617it [00:22, 18512.37it/s]

415469it [00:22, 18506.84it/s]

417320it [00:22, 18500.79it/s]

419176it [00:22, 18515.90it/s]

421028it [00:22, 18511.86it/s]

422880it [00:22, 18498.59it/s]

424734it [00:22, 18510.96it/s]

426586it [00:22, 18510.82it/s]

428438it [00:23, 18503.18it/s]

430292it [00:23, 18512.81it/s]

432144it [00:23, 18506.78it/s]

434001it [00:23, 18522.97it/s]

435854it [00:23, 18523.76it/s]

437710it [00:23, 18533.74it/s]

439564it [00:23, 18530.17it/s]

441418it [00:23, 18523.34it/s]

443272it [00:23, 18527.29it/s]

445125it [00:23, 18509.23it/s]

446978it [00:24, 18514.21it/s]

448830it [00:24, 18515.66it/s]

450684it [00:24, 18520.70it/s]

452541it [00:24, 18533.53it/s]

454396it [00:24, 18534.43it/s]

456250it [00:24, 18532.07it/s]

458104it [00:24, 18529.67it/s]

459957it [00:24, 18520.98it/s]

461810it

801932it [00:43, 18421.24it/s]

803776it [00:43, 18426.23it/s]

805620it [00:43, 18427.91it/s]

807464it [00:43, 18429.04it/s]

809307it [00:43, 18427.53it/s]

811150it [00:43, 18424.33it/s]

812994it [00:43, 18427.59it/s]

814839it [00:43, 18431.12it/s]

816684it [00:44, 18432.43it/s]

818528it [00:44, 18434.32it/s]

820372it [00:44, 18415.59it/s]

822215it [00:44, 18418.18it/s]

824057it [00:44, 18393.00it/s]

825898it [00:44, 18396.79it/s]

827744it [00:44, 18413.00it/s]

829586it [00:44, 18394.71it/s]

831427it [00:44, 18396.73it/s]

833268it [00:44, 18399.57it/s]

835109it [00:45, 18399.98it/s]

836954it [00:45, 18413.33it/s]

838796it [00:45, 18400.59it/s]

840638it [00:45, 18403.51it/s]

842480it [00:45, 18407.85it/s]

844324it [00:45, 18416.02it/s]

846166it [00:45, 18412.12it/s]

848008it [00:45, 18406.32it/s]

849850it [00:45, 18409.65it/s]

851691it [00:45, 18409.64it/s]

853533it [00:46, 18412.25it/s]

855377it [00:46, 18419.27it/s]

857220it [00:46, 18421.28it/s]

859064it

1192439it [01:04, 18353.92it/s]

1194276it [01:04, 18356.29it/s]

1196116it [01:04, 18368.88it/s]

1197958it [01:04, 18383.06it/s]

1199799it [01:04, 18389.89it/s]

1201638it [01:05, 18387.55it/s]

1203481it [01:05, 18396.88it/s]

1205321it [01:05, 18396.95it/s]

1207165it [01:05, 18407.54it/s]

1209006it [01:05, 18405.42it/s]

1210849it [01:05, 18411.62it/s]

1212693it [01:05, 18419.94it/s]

1214535it [01:05, 18396.63it/s]

1216379it [01:05, 18407.51it/s]

1218224it [01:05, 18418.64it/s]

1220066it [01:06, 18414.84it/s]

1221908it [01:06, 18403.61it/s]

1223749it [01:06, 18400.16it/s]

1225590it [01:06, 18389.89it/s]

1227429it [01:06, 18386.17it/s]

1229269it [01:06, 18386.19it/s]

1231109it [01:06, 18388.73it/s]

1232951it [01:06, 18395.89it/s]

1234794it [01:06, 18403.75it/s]

1236635it [01:06, 18386.66it/s]

1238474it [01:07, 18370.56it/s]

1240314it [01:07, 18377.23it/s]

1242153it [01:07, 18379.27it/s]

1243994it [01:07, 18387.96it/s]

1245833it [01:07, 18383.25it/s]

1247676it 

1579024it [01:25, 18398.39it/s]

1580867it [01:25, 18406.06it/s]

1582708it [01:25, 18404.45it/s]

1584554it [01:25, 18419.79it/s]

1586396it [01:25, 18405.43it/s]

1588237it [01:26, 18402.48it/s]

1590078it [01:26, 18391.43it/s]

1591918it [01:26, 18381.82it/s]

1593757it [01:26, 18381.70it/s]

1595599it [01:26, 18392.64it/s]

1597439it [01:26, 18388.81it/s]

1599281it [01:26, 18396.08it/s]

1601121it [01:26, 18374.48it/s]

1602959it [01:26, 18371.36it/s]

1604797it [01:26, 18372.46it/s]

1606636it [01:27, 18377.39it/s]

1608476it [01:27, 18381.52it/s]

1610315it [01:27, 18281.91it/s]

1612144it [01:27, 18229.65it/s]

1613968it [01:27, 18192.81it/s]

1615788it [01:27, 18169.42it/s]

1617606it [01:27, 18145.38it/s]

1619421it [01:27, 18091.22it/s]

1621236it [01:27, 18106.99it/s]

1623047it [01:27, 18107.19it/s]

1624858it [01:28, 18103.80it/s]

1626669it [01:28, 18104.13it/s]

1628480it [01:28, 18055.11it/s]

1630288it [01:28, 18059.47it/s]

1632102it [01:28, 18079.86it/s]

1633918it 

1959841it [01:46, 18066.34it/s]

1961650it [01:46, 18073.19it/s]

1963461it [01:46, 18081.64it/s]

1965270it [01:46, 18062.28it/s]

1967079it [01:47, 18068.76it/s]

1968886it [01:47, 18059.45it/s]

1970692it [01:47, 18049.25it/s]

1972499it [01:47, 18054.88it/s]

1974309it [01:47, 18065.99it/s]

1976117it [01:47, 18067.52it/s]

1977925it [01:47, 18068.99it/s]

1979732it [01:47, 18045.49it/s]

1981537it [01:47, 18032.67it/s]

1983341it [01:47, 18019.92it/s]

1985153it [01:48, 18048.18it/s]

1986960it [01:48, 18052.38it/s]

1988769it [01:48, 18061.54it/s]

1990576it [01:48, 18050.21it/s]

1992382it [01:48, 18045.33it/s]

1994190it [01:48, 18053.85it/s]

1995999it [01:48, 18064.07it/s]

1997814it [01:48, 18087.61it/s]

1999623it [01:48, 18068.40it/s]

2001430it [01:48, 18063.64it/s]

2003241it [01:49, 18075.96it/s]

2005049it [01:49, 18069.23it/s]

2006857it [01:49, 18072.43it/s]

2008665it [01:49, 18063.54it/s]

2010472it [01:49, 18064.21it/s]

2012279it [01:49, 18050.29it/s]

2014085it 

IndexError: index 1129 is out of bounds for axis 0 with size 1129

In [64]:
a = np.arange(16).reshape(4,4)
b = np.zeros(4)+2
print(a)
print(b)
np.dot(a,b)

[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]
 [12 13 14 15]]
[ 2.  2.  2.  2.]


array([  12.,   44.,   76.,  108.])